In [34]:
from dash import Dash, html, dcc, Input, Output  # pip install dash
import plotly.express as px
import dash_ag_grid as dag                       # pip install dash-ag-grid
import dash_bootstrap_components as dbc          # pip install dash-bootstrap-components
import pandas as pd                              # pip install pandas

import matplotlib                                # pip install matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import base64
from io import BytesIO

import requests
import numpy as np

import plotly.graph_objects as go

In [95]:
df = pd.read_csv("spese.csv", quotechar="'")

selected_columns = ['Territorio', 'COICOP', 'Coicop (DESC)', 'componenti', 'Anno', 'Spesa media']
df = df[selected_columns]

#spesa media
df['Spesa media'] = pd.to_numeric(df['Spesa media'], errors='coerce')
df['Spesa media'] = df['Spesa media'].replace(np.nan, -1, regex=True)
df['Spesa media'] = df['Spesa media'].astype(float)
df = df[df['Spesa media'] >= 0]

#territorio
df.replace(inplace=True, to_replace='Valle d"Aosta / Vallée d"\'Aoste\'', value='Valle d\'Aosta/Vallée d\'Aoste')
df.replace(inplace=True, to_replace='Trentino Alto Adige / Südtirol', value='Trentino-Alto Adige/Südtirol')

#categorie
values_to_exclude = ['0220', '0461_MANUT_STR', '0520', '0540', '0630', '0810', '0820', '0830', '0960', '1010', '1020', '1030', '1040', '1050', '1120', '1240', '1262', '1270']
df = df[~df['COICOP'].isin(values_to_exclude)]
df.loc[df['COICOP'] == '046_MANUT_STR', 'COICOP'] = '046'

#root
df.loc[df['COICOP'] == 'NON_FOOD', 'Root'] = 'Non alimenti'
df.loc[((df['COICOP'].str.len() == 2)) | (df['COICOP'] == 'ALL'), 'Root'] = df['Coicop (DESC)']

##first level
df.loc[((df['COICOP'].str.len() == 3) & (df['Root'].isna())), 'First_level'] = df['Coicop (DESC)']

##second level
df.loc[((df['COICOP'].str.len() == 4) & (df['First_level'].isna()) & (df['Root'].isna())), 'Second_level'] = df['Coicop (DESC)']

#fill the gaps
mask = df['Root'].isna()
df['Root'] = df['Root'].fillna(method='ffill')
df.loc[mask, 'First_level'] = df['First_level'].fillna(method='ffill')

macro_category=df[df['First_level'].isna() & df['Second_level'].isna()]['Coicop (DESC)'].unique()

In [97]:
df['Territorio'].unique()

array(['Italia', 'Nord', 'Piemonte', 'Valle d"Aosta / Vallée d"\'Aoste\'',
       'Liguria', 'Lombardia', 'Trentino-Alto Adige/Südtirol',
       'Provincia Autonoma Bolzano / Bozen', 'Provincia Autonoma Trento',
       'Veneto', 'Friuli-Venezia Giulia', 'Emilia-Romagna', 'Centro',
       'Toscana', 'Umbria', 'Marche', 'Lazio', 'Mezzogiorno', 'Abruzzo',
       'Molise', 'Campania', 'Puglia', 'Basilicata', 'Calabria',
       'Sicilia', 'Sardegna'], dtype=object)

In [77]:
region = 'Italia'
components = '1'
value_list = [str(i) for i in range(1, 9)]
mask = df[(df['Territorio'] == region) & (df['componenti'] == components) & (df['COICOP'].isin(value_list))]
mask = mask.sort_values(by=['Anno', 'Spesa media'])
display(mask)

,Territorio,COICOP,Coicop (DESC),componenti,Anno,Spesa media,Root,First_level,Second_level
138,Italia,2,Bevande alcoliche e tabacchi,1,2022,30.75,Non alimenti,Servizi per la trasformazione delle materie pr...,NaN
732,Italia,8,Informazione e comunicazione,1,2022,49.13,Non alimenti,Servizi di trasporto merci,NaN
198,Italia,3,Abbigliamento e calzature,1,2022,61.84,Non alimenti,Tabacchi,NaN
390,Italia,5,"Mobili, articoli e servizi per la casa",1,2022,76.89,Non alimenti,Interventi di ristrutturazione,NaN
510,Italia,6,Salute,1,2022,80.64,Non alimenti,Beni e servizi per la manutenzione ordinaria d...,NaN
606,Italia,7,Trasporti,1,2022,152.42,Non alimenti,Altri servizi sanitari,NaN
6,Italia,1,Prodotti alimentari e bevande analcoliche,1,2022,313.52,Totale,NaN,NaN
252,Italia,4,"Abitazione, acqua, elettricità, gas e altri co...",1,2022,889.97,Non alimenti,Calzature,NaN


In [78]:
value_list = [str(i) for i in range(1, 10)]
region = 'Italia'
year=2020
mask = df[(df['Territorio'] == region) & (df['Anno'] == year) & (df['COICOP'].isin(value_list))]

grouped_data = mask.groupby(['componenti', 'Coicop (DESC)']).agg(mean_value=('Spesa media', 'mean')).reset_index()
mean_values = grouped_data.groupby(['Coicop (DESC)']).agg(mean_value=('mean_value', 'mean')).reset_index()

display(grouped_data)
display(mean_values)

,componenti,Coicop (DESC),mean_value


,Coicop (DESC),mean_value


In [79]:
value_list = [str(i) for i in range(1, 2)]
components = '1'
year=2020

mask = df[(df['componenti'] == components) & (df['Anno'] == year) & (df['COICOP'].isin(value_list))]
geojson_italy = 'https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_regions.geojson'
#italy_regions_geo = requests.get(repo_url).json()
#display(mask)
# Choropleth representing the length of region names
fig = px.choropleth(mask, 
                    geojson=geojson_italy, 
                    locations='Territorio', # name of dataframe column
                    featureidkey='properties.reg_name',  # path to field in GeoJSON feature object with which to match the values passed in to locations
                    color='Spesa media',
                    color_continuous_scale="Magma",
                    scope="europe",
                    labels={'Spesa media':'Spesa media'}
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [32]:
category = '-- trasporti'
components = '1'
year=2020

macro_region = df[(df['componenti'] == components) & (df['Anno'] == year) & (df['Coicop (DESC)']==category) 
                      & (df['Territorio'].isin(['Nord','Mezzogiorno', 'Centro']))]
display(macro_region)

,Territorio,COICOP,Coicop (DESC),componenti,Anno,Spesa media
9086,Nord,7,-- trasporti,1,2020,143.39
15941,Centro,7,-- trasporti,1,2020,128.33
19436,Mezzogiorno,7,-- trasporti,1,2020,79.85


In [48]:
df['COICOP'].unique()


array(['1', '11', '111', '112', '113', '114', '115', '116', '117', '118',
       '119', '12', '121', '122', 'NON_FOOD', '2', '21', '211', '212',
       '213', '22', '3', '31', '311', '312', '313', '314', '32', '321',
       '322', '4', '41', '411', '412', '42', '421', '422', '43', '431',
       '432', '44', '441', '442', '443', '444', '45', '451', '452', '453',
       '454', '455', '46', '5', '51', '511', '512', '513', '52', '53',
       '531', '532', '533', '54', '55', '551', '552', '56', '561', '562',
       '6', '61', '611', '612', '613', '62', '621', '622', '623', '63',
       '7', '71', '711', '712', '713', '72', '721', '722', '723', '724',
       '73', '731', '732', '733', '734', '735', '736', '8', '81', '82',
       '83', '9', '91', '911', '912', '913', '914', '915', '92', '922',
       '923', '93', '931', '932', '933', '934', '935', '94', '941', '942',
       '943', '95', '951', '952', '953', '954', '96', '10', '101', '102',
       '103', '104', '105', '1111', '1112', '1211', '

In [6]:
macro_category=df[df['First_level'].isna() & df['Second_level'].isna()]['Coicop (DESC)'].unique()
print(macro_category)

['-- prodotti alimentari e bevande analcoliche' 'Non alimentari'
 '-- bevande alcoliche e tabacchi' '-- abbigliamento e calzature'
 '-- abitazione, acqua, elettricità, gas e altri combustibili'
 '-- mobili, articoli e servizi per la casa'
 '-- servizi sanitari e spese per la salute' '-- trasporti'
 '-- comunicazioni' '-- ricreazione, spettacoli e cultura' '-- istruzione'
 '-- servizi ricettivi e di ristorazione' '-- altri beni e servizi'
 'Totale']


In [10]:
region = 'Italia'
year=2020
mask = df[(df['Territorio'] == region) & (df['Anno'] == year) & (df['Coicop (DESC)'].isin(macro_category))]
grouped_data = mask.groupby(['componenti', 'Coicop (DESC)']).agg(spesa_media=('Spesa media', 'mean')).reset_index()
mean_values = mask.groupby('Coicop (DESC)')['Spesa media'].mean().reset_index()

In [ ]:
for _, mean_row in mean_values.iterrows():
    print([mean_row['Spesa media']])

In [28]:
display(mean_values)

,Coicop (DESC),Spesa media
0,-- abbigliamento e calzature,113.248
1,"-- abitazione, acqua, elettricità, gas e altri...",905.466
2,-- altri beni e servizi,186.116
3,-- bevande alcoliche e tabacchi,46.846
4,-- comunicazioni,61.944
5,-- istruzione,20.208
6,"-- mobili, articoli e servizi per la casa",114.426
7,-- prodotti alimentari e bevande analcoliche,542.494
8,"-- ricreazione, spettacoli e cultura",109.214
9,-- servizi ricettivi e di ristorazione,90.224


In [29]:
fig = go.Figure() #= px.histogram(grouped_data, x='Coicop (DESC)', y='spesa_media', color='componenti', barmode='group')



for i, group_df in grouped_data.groupby('componenti'):
        fig.add_trace(go.Bar(
            x=group_df['Coicop (DESC)'], 
            y=group_df['spesa_media'], 
            name=str(i),
            width=0.2
        ))



fig.update_layout(
        barmode='group', 
        yaxis_title_text = 'Spesa media', 
        #bargroupgap=0.1, 
        #bargap=0.2, 
    ) 
fig.update_yaxes(type="log")

l = px.line(
    mean_values, x="Coicop (DESC)", y="Spesa media"
).update_traces(line_color="red")

fig.add_traces(l.data)

fig.show()

In [39]:
mask = df[(df['componenti'] == '1') & (df['Anno'] == 2020) & (df['Territorio']=='Italia')]
dff = mask.groupby(by=["Root", "First_level", "Second_level", "Coicop (DESC)"]).sum('Spesa media').reset_index()
fig = px.treemap(
    dff, path=[px.Constant("ALL"), "Root", "First_level", "Coicop (DESC)"], 
    values='Spesa media', labels={
            'ALL': 'All',
            'Root': 'Coicop (DESC)',
            'First_level': 'Coicop (DESC)',
            'Second_level': 'Coicop (DESC)'
        },
)

fig.show()

In [60]:
df.loc[df['COICOP'] == 'NON_FOOD', 'Root'] = 'Non alimenti'
df.loc[((df['COICOP'].str.len() <= 2) & (df['Coicop (DESC)'].str.startswith("--"))) | (df['COICOP'] == 'ALL'), 'Root'] = df['Coicop (DESC)']

##first level
df.loc[((df['COICOP'].str.len() == 2) & (df['Root'].isna())) |
    ((df['COICOP'].str.len() == 2) & (df['COICOP'].isin([111, 112])) 
     & (df['Coicop (DESC)'].isin(['Pane e cereali', 'Carni']))), 'First_level'] = df['Coicop (DESC)']#.apply(lambda x: x[-1])

##second level
df.loc[((df['COICOP'].str.len() == 3) & (df['First_level'].isna()) & (df['Root'].isna())) |
       ((df['COICOP'].str.len() == 4)), 'Second_level'] = df['Coicop (DESC)']#.apply(lambda x: x[-1])

#fill the gaps
mask = df['Root'].isna()
df['Root'] = df['Root'].fillna(method='ffill')
df.loc[mask, 'First_level'] = df['First_level'].fillna(method='ffill')

In [63]:
display(df['Root'].unique())

array(['-- prodotti alimentari e bevande analcoliche', 'Non alimenti',
       '-- bevande alcoliche e tabacchi', '-- abbigliamento e calzature',
       '-- abitazione, acqua, elettricità, gas e altri combustibili',
       '-- mobili, articoli e servizi per la casa',
       '-- servizi sanitari e spese per la salute', '-- trasporti',
       '-- comunicazioni', '-- ricreazione, spettacoli e cultura',
       '-- istruzione', '-- servizi ricettivi e di ristorazione',
       '-- altri beni e servizi', 'Totale'], dtype=object)

In [70]:
mask = df[(df['componenti'] == '1') & (df['Anno'] == 2020) & (df['Territorio']=='Italia')]
dff = mask.groupby(by=["Root", "First_level", "Second_level"]).sum('Spesa media').reset_index()
fig = px.treemap(
    dff, path=[px.Constant("ALL"), "Root", "First_level", "Second_level"], 
    values='Spesa media', labels={
        'id': 'Coicop (DESC)'
    }
)
fig.data[0].textinfo = 'label+text+value'
fig.show()

In [75]:
df = pd.read_csv("spese-2022.csv", quotechar="'")
selected_columns = ['Territorio', 'COICOP', 'Coicop (DESC)', 'componenti', 'Anno', 'Spesa media']
df = df[selected_columns]

#spesa media
df['Spesa media'] = pd.to_numeric(df['Spesa media'], errors='coerce')
df['Spesa media'] = df['Spesa media'].replace(np.nan, -1, regex=True)
df['Spesa media'] = df['Spesa media'].astype(float)
df = df[df['Spesa media'] >= 0]

#territorio
df.replace(inplace=True, to_replace='Valle d"\'Aosta / Vallée d"\'Aoste', value='Valle d\'Aosta/Vallée d\'Aoste')
df.replace(inplace=True, to_replace='Trentino Alto Adige / Südtirol', value='Trentino-Alto Adige/Südtirol')

#categorie
values_to_exclude = ['130', '220', '0461_MAJOR_REN', '540', '820', '980', '1010', '1020', '1030', '1040', '1050', '1120', '1330', '1390']

df = df[~df['COICOP'].isin(values_to_exclude)]
df.loc[df['COICOP'] == '046_MAJOR_REN', 'COICOP'] = '46'

df.loc[df['COICOP'] == 'NON_FOOD', 'Root'] = 'Non alimenti'
df.loc[((df['COICOP'].str.len() <= 2) & (df['Coicop (DESC)'].str.startswith("--"))) | (df['COICOP'] == 'ALL'), 'Root'] = df['Coicop (DESC)']

##first level
df.loc[((df['COICOP'].str.len() == 2) & (df['Root'].isna())) |
    ((df['COICOP'].str.len() == 2) & (df['COICOP'].isin([111, 112])) 
     & (df['Coicop (DESC)'].isin(['Pane e cereali', 'Carni']))), 'First_level'] = df['Coicop (DESC)']

##second level
df.loc[((df['COICOP'].str.len() == 3) & (df['First_level'].isna()) & (df['Root'].isna())) |
       ((df['COICOP'].str.len() == 4)), 'Second_level'] = df['Coicop (DESC)']

#fill the gaps
mask = df['Root'].isna()
df['Root'] = df['Root'].fillna(method='ffill')
df.loc[mask, 'First_level'] = df['First_level'].fillna(method='ffill')

In [73]:
df['Territorio'].unique()

array(['Italia', 'Nord', 'Piemonte', "Valle d'Aosta/Vallée d'Aoste",
       'Liguria', 'Lombardia', 'Trentino-Alto Adige/Südtirol',
       'Provincia Autonoma Bolzano / Bozen', 'Provincia Autonoma Trento',
       'Veneto', 'Friuli-Venezia Giulia', 'Emilia-Romagna', 'Centro',
       'Toscana', 'Umbria', 'Marche', 'Lazio', 'Mezzogiorno', 'Abruzzo',
       'Molise', 'Campania', 'Puglia', 'Basilicata', 'Calabria',
       'Sicilia', 'Sardegna'], dtype=object)